In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models

import numpy as np
import datetime

In [37]:
# hyperparameter setting
learning_rate = 0.0001
num_epochs = 30
batch_size = 512

# device detection (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [38]:
class ImageDataset(Dataset):
    '''
    Dataset class for loading and processing images and labels from NumPy arrays.
    '''

    def __init__(self, images, labels, transforms):
        '''
        Initialization of the dataset.

        Args:
            Images (numpy.ndarray): Numpy array with images.
            labels (numpy.ndarray): NumPy array with labels.
            transforms (torchvision.transforms, optional): Transforms to apply to images.
        '''
        
        self.images = images
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        '''
        Returns the length of the dataset (number of images).
        '''
        
        return len(self.images)

    def __getitem__(self, index):
        '''
        Returns a single element from the dataset (image and label) at the given index.

        Args:
            index (int): The index of an element in the dataset.

        Returns:
            tuple: A tuple containing the transformed image and label.
        '''
        
        image = self.images[index]
        label = self.labels[index]

        # Apply transformations (if defined)
        if self.transforms is not None:
            image = self.transforms(image)

        if isinstance(image, np.ndarray):
            image = torch.from_numpy(image).float()

        if isinstance(label, np.ndarray):
            label = torch.from_numpy(label).long()

        return image, label


In [39]:
# load images and labels from .npy format to Numpy arrays
images = np.load('./images.npy')
labels = np.load('./labels.npy')

# define transforms
data_transforms = transforms.Compose([
    transforms.ToTensor(), # to torch tensor
    transforms.Grayscale() # to one channel
])

# defining and customizing the dataset
dataset = ImageDataset(images, labels, transforms=data_transforms)

# dividing the data set into training and test samples
train_data, test_data = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), int(0.2 * len(dataset))])

# defining and customizing the train and test dataloaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [40]:
# load CNN model ResNet18
model = models.resnet18(pretrained=False)

# setting number of classes
num_classes = 3

# setting the first convolution layer
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# number of input functions for the last layer
num_features = model.fc.in_features

# new fully connected layer
model.fc = nn.Linear(num_features, num_classes)

model.to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [41]:
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [42]:
# log setting
current_time = datetime.datetime.now()
log_file = f'./model_checkpoint/training_log{current_time.strftime("%Y-%m-%d_%H-%M-%S")}.txt'
model_save_path = f'./model_checkpoint/model_{current_time.strftime("%Y-%m-%d_%H-%M-%S")}.pt'

In [43]:
# training loop
for epoch in range(num_epochs):
    print(f"epoch {epoch + 1}/{num_epochs}")

    # set model to training mode
    model.train()
    
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        labels = labels.long()
        
        # gradient cleaning
        optimizer.zero_grad()
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward
        loss.backward()
        optimizer.step()

        # loss calculation
        running_loss += loss.item()

        if i % 100 == 99:
            average_loss = running_loss / 100
            print(f"[Batch {i + 1}/{len(train_loader)}] Loss: {average_loss:.4f}")
            running_loss = 0.0

    # validation
    model.eval()
    
    test_loss = 0.0
    accuracy = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.long()

            outputs = model(images)

            # calculation loss and accuracy on test data
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()

    test_loss /= len(test_loader)
    accuracy /= total

    # Recording data on training results to a log file
    with open(log_file, 'a') as f:
        f.write(f'test_loss: {test_loss:.4f} | test_accuracy: {accuracy:.4f}')
    
    print(f'test_loss: {test_loss:.4f} | test_accuracy: {accuracy:.4f}')


epoch 1/30
test_loss: 2.3550 | test_accuracy: 0.4011
epoch 2/30
test_loss: 3.1644 | test_accuracy: 0.4011
epoch 3/30
test_loss: 4.1389 | test_accuracy: 0.4017
epoch 4/30
test_loss: 3.7233 | test_accuracy: 0.4094
epoch 5/30
test_loss: 2.6399 | test_accuracy: 0.5312
epoch 6/30
test_loss: 1.0113 | test_accuracy: 0.7190
epoch 7/30
test_loss: 0.6281 | test_accuracy: 0.8111
epoch 8/30
test_loss: 0.8857 | test_accuracy: 0.7522
epoch 9/30
test_loss: 0.8071 | test_accuracy: 0.7796
epoch 10/30
test_loss: 0.5970 | test_accuracy: 0.8081
epoch 11/30
test_loss: 0.6772 | test_accuracy: 0.8111
epoch 12/30
test_loss: 0.7692 | test_accuracy: 0.8027
epoch 13/30
test_loss: 0.6352 | test_accuracy: 0.8206
epoch 14/30
test_loss: 0.6791 | test_accuracy: 0.8146
epoch 15/30
test_loss: 0.6546 | test_accuracy: 0.8200
epoch 16/30
test_loss: 0.6591 | test_accuracy: 0.8182
epoch 17/30
test_loss: 0.6563 | test_accuracy: 0.8158
epoch 18/30
test_loss: 0.7008 | test_accuracy: 0.8033
epoch 19/30
test_loss: 0.6816 | test_

In [44]:
# save model
torch.save(model, model_save_path)

In [45]:
# save model and optimizer
torch.save({'model': model, 'optimizer': optimizer}, (model_save_path + '_and_optim'))

In [46]:
# Saving the model state
torch.save(model.state_dict(), (model_save_path + 'state_dict'))